<a href="https://colab.research.google.com/github/estebanhernandezr/Entregas-Semanales/blob/main/Scripts_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **imports**

In [55]:
import psycopg2
import pandas as pd

### **functions**

In [218]:
def db_selection(what, where):
    engine = None
    resultDataFrame = None
    try:
        engine = psycopg2.connect(
            database = "UDriveDB",
            user = "UDriveAdmin",
            password = "UDriveAdmin10",
            host = "database-1.ct0kpzfxpmof.us-east-1.rds.amazonaws.com",
            port = "5432"
        )
        #cursorDB = engine.cursor()
        #cursorDB.execute("SELECT "+what+" FROM "+where+";")
        #resultDataFrame = cursorDB.fetchone()
        data = pd.read_sql_query("SELECT "+what+" FROM "+where+";", engine)
        resultDataFrame = pd.DataFrame(data)
    except(Exception, psycopg2.DatabaseError) as error:
        print("Error:", error)
        raise(ValueError(Exception))
    finally:
          if engine is not None:
              engine.close()
          return resultDataFrame

In [237]:
def count_column_values(dataframe, col):
    print(dataframe[col].value_counts())

### **main**

In [233]:
def main():
    usuariosDF = db_selection('*', 'Usuario')
    count_column_values(dataframe=usuariosDF, col='nombre')

In [235]:
main()

Michael Hernández    3
Oscar Velasco        2
Fidel Castro         1
juan Ladino          1
hebreo salidas       1
mario jimenez        1
Camila Fernández     1
julian jimenez       1
Alejandra Pardo      1
Daniel Ramirez       1
Sara Palacios        1
Name: nombre, dtype: int64


### **tests suite**

In [225]:
usuariosDF = db_selection('*', 'Usuario')
usuariosDF['nombre'].value_counts()

Michael Hernández    3
Oscar Velasco        2
Fidel Castro         1
juan Ladino          1
hebreo salidas       1
mario jimenez        1
Camila Fernández     1
julian jimenez       1
Alejandra Pardo      1
Daniel Ramirez       1
Sara Palacios        1
Name: nombre, dtype: int64